In [ ]:
# !git clone https://github.com/FelipeSchreiber/snap.git
# !cd snap;bash install_in_linux.sh

In [ ]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../src/')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import fsolve
import networkx as nx
from networkx.algorithms import bipartite
import subprocess
from scipy.stats import powerlaw, expon
import scipy.stats as stats
from attrAgmFit import *

In [ ]:
!python --version

### Agora geramos um grafo bipartido aleatoriamente
<a href="https://networkx.org/documentation/stable/reference/algorithms/bipartite.html">Referencia</a>

In [ ]:
k = 3
B = bipartite.random_graph(1000,k,0.15)
top_nodes = {n for n, d in B.nodes(data=True) if d["bipartite"] == 0}
bottom_nodes = set(B) - top_nodes
pos = nx.bipartite_layout(B, top_nodes)
nx.draw(B,pos=pos)

In [ ]:
bottom_nodes

In [ ]:
nx.is_connected(B)

In [ ]:
B.number_of_nodes()

In [ ]:
for u in range(B.number_of_nodes()):
    if u not in B.nodes:
        print(f"{u} not in B")

### Save to txt file. Each line contains the nodes belonging to one group

In [ ]:
with open("bipartite.txt",'w') as f:
    for node in bottom_nodes:
        group = [str(n) for n in B.neighbors(node)]
        str_group = ' '.join(group) + "\n"
        f.write(str_group)

In [ ]:
probs = [str(round(i,2)) for i in np.random.uniform(0,1,k)]
str_probs = ",".join(probs)
lambdas = [str(round(i,2)) for i in np.random.rand(k+1)]
str_lambdas = ",".join(lambdas)
## -i:input_filename, -a:powerlaw_coeff, -c:powerlaw_coeff, -o:output_filename
# subprocess.run(["./agmgen","-i:bipartite.txt", "-a:0.6", "-c:1.3", "-o:agm_net", "-l:%s"%str_lambdas, "-pn:-1"])
subprocess.run(["../src/agmgen","-i:bipartite.txt", "-a:0.6", "-c:1.3", "-o:agm_net.txt", "-l:%s"%str_lambdas, "-p:%s"%str_probs, "-pn:0.1"])

In [ ]:
G = nx.read_edgelist("agm_net.txt",create_using=nx.Graph, nodetype = int, edgetype=int, data=(("weight", float),))
#first_label is the starting integer label, in this case zero
G = nx.relabel.convert_node_labels_to_integers(G, first_label=0, ordering='default') 
# for u, v, d in G.edges(data=True):
#     print(u,v,d)

In [ ]:
for u in range(G.number_of_nodes()):
    if u not in G.nodes:
        print(f"{u} not in G")

### Fits bigclam algorithm

In [ ]:
import bigclam as bc
F_init = np.random.rand(G.number_of_nodes(),k)

In [ ]:
#F_1 = bc.train(G, k, iterations = 10, lr=0.005, display_loss=True, F_init=F_init.copy())
#F_2 = bc.train_efficient(G, k, iterations = 10, lr=0.005, display_loss=True, F_init=F_init.copy())
#F_3 = bc.matrix_factorization(G, k, iterations = 10, lr=0.005)
F = bc.train_efficient(G, k, iterations = 10, lr=0.005, display_loss=True, F_init=F_init.copy())

In [ ]:
for i in range(k):
    print(1-np.exp(-np.outer(F[:,i],F[:,i])).mean())

In [ ]:
for i in range(k):
    data = F[:,i]
    fig, ax = plt.subplots(1,1)
    x = np.linspace(1, data.max(),100)
    a,loc,scale = powerlaw.fit(data)
    powerlaw_rv = powerlaw(a,loc,scale)
    y = powerlaw_rv.pdf(x)
    ax.plot(x,y,label='powerlaw pdf')
    powerlaw_test = stats.kstest(data, stats.powerlaw.cdf, args=(a, loc, scale))

    loc,scale = expon.fit(data)
    exp_rv = expon(loc=loc,scale=scale)
    y = exp_rv.pdf(x)
    ax.plot(x,y,label='expon pdf')
    # evaluate lognormal distribution
    exp_test = stats.kstest(data, stats.expon.cdf, args=(loc,scale))
    if exp_test.statistic < powerlaw_test.statistic:
        print("Exp distribution is the best fit.",(loc,scale))
    else:
        print("Powerlaw distribution is the best fit.\n",(a, loc, scale),f"\nmean = {powerlaw_rv.mean()}, median = {powerlaw_rv.median()}")
    
    ax.hist(data, density=True, bins='auto', histtype='stepfilled', alpha=0.2,label="hist")
    ax.set_xlim([x[0], x[-1]])
    ax.set_ylim([0, 0.1])
    ax.legend(loc='best', frameon=False)
    
    plt.show()
    #sns.lineplot(x=x,y=y,ax=axs[0])
    #sns.histplot(data=F[:,i],stat="probability",ax=axs[0])

### Now we generate a random bipartite graph, with power law degree distribution

In [ ]:
k = 5
B_sizes = powerlaw.rvs(a=0.5,loc=20,scale=1000,size=k).astype(int)
print(B_sizes)
sns.displot(data=B_sizes,stat="probability")

In [ ]:
total = np.sum(B_sizes)
total_a = 0
A_sizes = np.random.randint(1,k,int(total/k))
while (total - total_a > k):
    A_sizes = np.append(A_sizes,[np.random.randint(1,k)])
    total_a = np.sum(A_sizes)
# B_sizes = powerlaw.rvs(a=0.3,loc=A_sizes.min(),scale=total,size=2).astype(int)
if A_sizes.sum() < B_sizes.sum():
    A_sizes = np.append(A_sizes,[total - np.sum(A_sizes)])

In [ ]:
A_sizes[-1]

In [ ]:
A_sizes.sum(),B_sizes.sum()

In [ ]:
assert np.sum(B_sizes) == np.sum(A_sizes)

In [ ]:
G = bipartite.configuration_model(A_sizes.astype(int),B_sizes.astype(int),create_using=nx.Graph() )
top_nodes = {n for n, d in G.nodes(data=True) if d["bipartite"] == 0}
bottom_nodes = set(G) - top_nodes
pos = nx.bipartite_layout(G, top_nodes)
nx.draw(G,pos=pos)

In [ ]:
bottom_nodes

In [ ]:
with open("bipartite_powerlaw.txt",'w') as f:
    for node in bottom_nodes:
        group = [str(n) for n in G.neighbors(node)]
        str_group = ' '.join(group) + "\n"
        f.write(str_group)

In [ ]:
probs = [str(round(i,2)) for i in np.random.rand(k)]
str_probs = ",".join(probs)
lambdas = [str(round(i,2)) for i in np.random.rand(k)]
str_lambdas = ",".join(lambdas)
## -i:input_filename, -a:powerlaw_coeff, -c:powerlaw_coeff, -o:output_filename
subprocess.run(["../src/agmgen","-i:bipartite_powerlaw.txt", "-a:0.6", "-c:1.3", "-o:agm_net.txt", "-l:%s"%str_lambdas, "-p:%s"%str_probs])

In [ ]:
G = nx.read_edgelist("agm_net.txt",create_using=nx.Graph, nodetype = int, edgetype=int, data=(("weight", float),))
G = nx.relabel.convert_node_labels_to_integers(G, first_label=0, ordering='default') 

In [ ]:
F = bc.train_efficient(G, k, iterations = 3)

In [ ]:
for i in range(k):
    data = F[:,i]
    fig, ax = plt.subplots(1,1)
    x = np.linspace(1, data.max(),100)
    a,loc,scale = powerlaw.fit(data)
    powerlaw_rv = powerlaw(a,loc,scale)
    y = powerlaw_rv.pdf(x)
    ax.plot(x,y,label='powerlaw pdf')
    powerlaw_test = stats.kstest(data, stats.powerlaw.cdf, args=(a, loc, scale))

    loc,scale = expon.fit(data)
    exp_rv = expon(loc=loc,scale=scale)
    y = exp_rv.pdf(x)
    ax.plot(x,y,label='expon pdf')
    # evaluate lognormal distribution
    exp_test = stats.kstest(data, stats.expon.cdf, args=(loc,scale))
    if exp_test.statistic < powerlaw_test.statistic:
        print("Exp distribution is the best fit.",(loc,scale))
    else:
        print("Powerlaw distribution is the best fit.\n",(a, loc, scale),f"\nmean = {powerlaw_rv.mean()}, median = {powerlaw_rv.median()}")
    
    ax.hist(data, density=True, bins='auto', histtype='stepfilled', alpha=0.2,label="hist")
    ax.set_xlim([x[0], x[-1]])
    ax.set_ylim([0, 0.8])
    ax.legend(loc='best', frameon=False)
    
    plt.show()
    #sns.lineplot(x=x,y=y,ax=axs[0])
    #sns.histplot(data=F[:,i],stat="probability",ax=axs[0])

In [ ]:
from attrAgm import *
k = 3
WC = np.random.rand(2*k,4)
benchmark_instance = attrAGM(weight_centers=WC)
G,Y,E,M = benchmark_instance.generate_benchmark(n_steps=10)
nx.write_weighted_edgelist(G,'net.txt',delimiter=',')

In [ ]:
F_init = bc.train_efficient(G, k, iterations = 40, display_loss=True, use_line_search=True)

In [ ]:
F_init

In [ ]:
# from attrAgmFit import *
A = benchmark_instance.radius * benchmark_instance.att_centers
estimator  = attrAgmFit( MinVal=0,
                         MaxVal=1000,
                         num_communities = k,
                         update_F_iterations = 1,
                         update_F_lr=0.005,
                         display_loss=True, 
                         use_line_search=False,
                         line_search_Alpha=0.05,
                         line_search_Beta=0.3,
                         line_search_MaxIter=5
                        )

In [ ]:
F,A,B = estimator.fit_MLE(G,Y,iterations=15, F_init=F_init)

In [ ]:
F.shape,A.shape,B.shape

In [ ]:
F

In [ ]:
plt.scatter(A[:,0],A[:,1])

In [ ]:
attrAGM().gen_bipartite_net(num_nodes=300,n_clusters=3,bipartite_prob=0.25)
G,_,M = attrAGM().generate_network("bipartite.txt","agm.txt")
G,Y,E,M = attrAGM().generate_benchmark(n_steps=15)

In [ ]:
M[:5,:]

In [ ]:
for i in range(5):
    print(G.nodes[i]["membership"])

In [ ]:
plt.scatter(Y[:,0],Y[:,1])

In [ ]:
G.edges[0,1]

In [ ]:
pi = np.ones(F.shape[1])*np.random.rand(3)

In [ ]:
# %%timeit
# res = (pi**F)*((1-pi)**(1-F))

In [ ]:
# %%timeit
# res2 = np.zeros(F.shape)
# for i in range(res2.shape[0]):
#     for h in range(res2.shape[1]):
#         res2[i][h] = (pi[h]**F[i][h])*((1-pi[h])**(1-F[i,h]))
# #(res == res2).all()

In [ ]:
from banerjee_overlapping import *
from scipy.spatial import distance
M,A,alphas = banerjee_overlapping(dist_func = distance.sqeuclidean).fit(Y,iterations=10,n_clus=3)

In [ ]:
plt.scatter(A[:,0],A[:,1])

In [ ]:
M,A,alphas = banerjee_overlapping(dist_func = distance.euclidean).fit_relaxed(Y,iterations=10,n_clus=3)
plt.scatter(A[:,0],A[:,1])

In [ ]:
M.mean(axis=0),(M>=0).all(),(M<=1).all()

In [ ]:
"""
Optionally uncomment bellow lines for reading from a file
"""
M = np.load('membership_matrix.npy')
Y = np.load('attr.npy')
df = pd.read_csv('net.txt',names=["u","v","w"])
G = nx.Graph()
for index, row in df.iterrows():
    G.add_edge(row["u"],row["v"])
    list_str_w = row["w"][1:-1].strip(" ").split(" ")
    G[row.u][row.v]["weight"] = np.array([x for x in list_str_w if x]).astype(np.float64)
df.head()

In [ ]:
G.number_of_nodes(),Y.shape[0],M.shape[0]

In [ ]:
np.random.rand(10).reshape((5,2)).sum(axis=0)

In [ ]:
# %%timeit
"""
PYTHON TAKES 19.3 s ± 374 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
JULIA TAKES 3.422 s ± 23.851 ms
"""
estimator  = attrAgmFit( MinVal=0,
                         MaxVal=1000,
                         num_communities = 3,
                         update_F_iterations = 1,
                         update_F_lr=0.005,
                         display_loss=True, 
                         use_line_search=False,
                         line_search_Alpha=0.05,
                         line_search_Beta=0.3,
                         line_search_MaxIter=5
                        )
F,A,B = estimator.fit_MLE(G,Y,iterations=15)